# ***6530611033 Project ML ข้อที่ 2***


### Google Drive Authentication

In [ ]:
# Access to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Import Library

In [ ]:
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from PIL import Image
import os
import shutil
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image, image_dataset_from_directory

In [ ]:
# เช็คไฟล์ว่ามีการเข้าถึงไหม
_base_dir = '/content/drive/MyDrive/Colab Notebooks/project_baselearning/photo'
print(os.listdir(_base_dir))

['No disease', 'Anthracnose', 'Rust fungus', 'testing']


### Prepare Data Directories

In [ ]:
# Define the base directory where your images are stored
_base_dir = '/content/drive/MyDrive/Colab Notebooks/project_baselearning/photo'
categories = ['Anthracnose', 'No disease', 'Rust fungus']
split_ratios = {'train': 0.7, 'val': 0.2, 'test': 0.1}

# Define output directories for Train, Val, and Test sets
output_dirs = {
    'train': '/content/drive/MyDrive/Colab Notebooks/project_baselearning/photo/testing/train',
    'val': '/content/drive/MyDrive/Colab Notebooks/project_baselearning/photo/testing/val',
    'test': '/content/drive/MyDrive/Colab Notebooks/project_baselearning/photo/testing/test'
}

In [ ]:
base_dir = '/content/drive/MyDrive/Colab Notebooks/project_baselearning/photo/testing'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')

### Model Initialization

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# กำหนดการ Augmentation สำหรับข้อมูลฝึก
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    horizontal_flip=True
)

# กำหนดการ Augmentation สำหรับข้อมูล validation
val_datagen = ImageDataGenerator(
    rescale=1.0/255
)

# โหลดข้อมูลฝึกจาก train_dir
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),  # ขนาดภาพที่ใช้กับโมเดล
    batch_size=32,
    class_mode='categorical'
)

# โหลดข้อมูล validation จาก val_dir
validation_generator = val_datagen.flow_from_directory(
    val_dir,  # ใช้ val_dir สำหรับข้อมูล validation
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical'
)

# โหลดข้อมูลทดสอบจาก test_dir
test_generator = val_datagen.flow_from_directory(
    test_dir,  # ใช้ test_dir สำหรับข้อมูลทดสอบ
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)


Found 715 images belonging to 3 classes.
Found 204 images belonging to 3 classes.
Found 104 images belonging to 3 classes.


# InceptionV3

In [ ]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout # Import necessary layers
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Load the pre-trained InceptionV3 model without the top classification layer
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(len(categories), activation='softmax')(x)  # Output layer with softmax

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Print the model summary
model.summary()

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2             │ (None, 128, 128, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 63, 63, 32)     │            864 │ input_layer_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization       │ (None, 63, 63, 32)     │             96 │ conv2d[0][0]           │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation (Activation)   │ (None, 63, 63, 32)     │              0 │ batch_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 61, 61, 32)     │          9,216 │ activation[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_1     │ (None, 61, 61, 32)     │             96 │ conv2d_1[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_1 (Activation) │ (None, 61, 61, 32)     │              0 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 61, 61, 64)     │         18,432 │ activation_1[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_2     │ (None, 61, 61, 64)     │            192 │ conv2d_2[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_2 (Activation) │ (None, 61, 61, 64)     │              0 │ batch_normalization_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d             │ (None, 30, 30, 64)     │              0 │ activation_2[0][0]     │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, 30, 30, 80)     │          5,120 │ max_pooling2d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_3     │ (None, 30, 30, 80)     │            240 │ conv2d_3[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_3 (Activation) │ (None, 30, 30, 80)     │              0 │ batch_normalization_3… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_4 (Conv2D)         │ (None, 28, 28, 192)    │        138,240 │ activation_3[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_4     │ (None, 28, 28, 192)    │            576 │ conv2d_4[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├──────────────────────

 Total params: 23,904,035 (91.19 MB)

 Trainable params: 2,101,251 (8.02 MB)

 Non-trainable params: 21,802,784 (83.17 MB)

In [ ]:
# กำหนดจำนวน steps และ batch size ให้เหมาะสม
batch_size = 64  # เพิ่ม batch size เป็น 64 เพื่อลดจำนวน batch ที่ต้องรัน

# ฝึกโมเดลโดยกำหนด steps_per_epoch และ validation_steps
history = model.fit(
    train_generator,
    steps_per_epoch=10,  # กำหนด steps ต่อ epoch เพื่อลดเวลาฝึก
    epochs=100,
    validation_data=validation_generator,
    validation_steps=5,  # กำหนด validation steps
)

Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


10/10 ━━━━━━━━━━━━━━━━━━━━ 186s 15s/step - accuracy: 0.4323 - loss: 1.2859 - val_accuracy: 0.7312 - val_loss: 0.6283
Epoch 2/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 61s 6s/step - accuracy: 0.7265 - loss: 0.6591 - val_accuracy: 0.8409 - val_loss: 0.4192
Epoch 3/100


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


10/10 ━━━━━━━━━━━━━━━━━━━━ 51s 5s/step - accuracy: 0.7099 - loss: 0.6056 - val_accuracy: 0.8125 - val_loss: 0.4499
Epoch 4/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 161s 9s/step - accuracy: 0.8472 - loss: 0.3956 - val_accuracy: 0.7955 - val_loss: 0.3927
Epoch 5/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 95s 10s/step - accuracy: 0.8223 - loss: 0.4414 - val_accuracy: 0.8875 - val_loss: 0.3274
Epoch 6/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 39s 4s/step - accuracy: 0.8651 - loss: 0.3121 - val_accuracy: 0.8409 - val_loss: 0.3273
Epoch 7/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 157s 13s/step - accuracy: 0.8796 - loss: 0.3018 - val_accuracy: 0.9187 - val_loss: 0.2635
Epoch 8/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 61s 6s/step - accuracy: 0.8945 - loss: 0.2572 - val_accuracy: 0.9091 - val_loss: 0.2826
Epoch 9/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 54s 5s/step - accuracy: 0.8422 - loss: 0.3682 - val_accuracy: 0.9312 - val_loss: 0.2516
Epoch 10/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 174s 8s/step - accuracy: 0.8901 - loss: 0.3248 - val_accuracy: 0.9091 - val_los

In [ ]:
loss, accuracy = model.evaluate(validation_generator)
print(f"Validation Accuracy: {accuracy:.2f}")

7/7 ━━━━━━━━━━━━━━━━━━━━ 46s 6s/step - accuracy: 0.9417 - loss: 0.1294
Validation Accuracy: 0.92


# **VGG16**

In [ ]:
from tensorflow.keras.applications import InceptionV3, VGG16 # Import VGG16

base_model = VGG16(input_shape=(128, 128, 3), include_top=False, weights='imagenet')
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(3, activation='softmax')(x)

model_VGG16 = Model(inputs=base_model.input, outputs=x)

for layer in base_model.layers:
    layer.trainable = False

model_VGG16.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


In [ ]:
# ฝึกโมเดลโดยกำหนด steps_per_epoch และ validation_steps
history = model_VGG16.fit(
    train_generator,
    steps_per_epoch=10,  # กำหนด steps ต่อ epoch เพื่อลดเวลาฝึก
    epochs=100,
    validation_data=validation_generator,
    validation_steps=5,  # กำหนด validation steps
)

Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


10/10 ━━━━━━━━━━━━━━━━━━━━ 229s 20s/step - accuracy: 0.3894 - loss: 1.1558 - val_accuracy: 0.6125 - val_loss: 0.8294
Epoch 2/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 91s 9s/step - accuracy: 0.5858 - loss: 0.9153 - val_accuracy: 0.7955 - val_loss: 0.6733
Epoch 3/100


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


10/10 ━━━━━━━━━━━━━━━━━━━━ 86s 9s/step - accuracy: 0.6594 - loss: 0.7978 - val_accuracy: 0.8375 - val_loss: 0.6206
Epoch 4/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 204s 14s/step - accuracy: 0.6832 - loss: 0.7749 - val_accuracy: 0.7500 - val_loss: 0.6262
Epoch 5/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 139s 14s/step - accuracy: 0.7637 - loss: 0.6335 - val_accuracy: 0.8438 - val_loss: 0.4894
Epoch 6/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 85s 9s/step - accuracy: 0.8203 - loss: 0.5296 - val_accuracy: 0.7955 - val_loss: 0.5198
Epoch 7/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 216s 19s/step - accuracy: 0.8353 - loss: 0.5425 - val_accuracy: 0.8625 - val_loss: 0.4273
Epoch 8/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 88s 9s/step - accuracy: 0.8040 - loss: 0.5143 - val_accuracy: 0.7045 - val_loss: 0.5373
Epoch 9/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 78s 8s/step - accuracy: 0.7766 - loss: 0.5055 - val_accuracy: 0.8313 - val_loss: 0.4336
Epoch 10/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 204s 17s/step - accuracy: 0.8399 - loss: 0.4691 - val_accuracy: 0.8182 - val_

In [ ]:
loss, accuracy = model_VGG16.evaluate(validation_generator)
print(f"Validation Accuracy: {accuracy:.2f}")

7/7 ━━━━━━━━━━━━━━━━━━━━ 68s 9s/step - accuracy: 0.9071 - loss: 0.2549
Validation Accuracy: 0.91


# **DenseNet121**

In [ ]:
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout # Import necessary layers
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

base_model = DenseNet121(input_shape=(128, 128, 3), include_top=False, weights='imagenet')
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(3, activation='softmax')(x)

model_DenseNet121 = Model(inputs=base_model.input, outputs=x)

for layer in base_model.layers:
    layer.trainable = False

model_DenseNet121.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model (using the existing train_generator and validation_generator)
history = model_DenseNet121.fit(
    train_generator,
    steps_per_epoch=2,
    epochs=100,
    validation_data=validation_generator,
    validation_steps=5
)

In [ ]:
loss, accuracy = model_DenseNet121.evaluate(validation_generator)
print(f"Validation Accuracy: {accuracy:.2f}")

7/7 ━━━━━━━━━━━━━━━━━━━━ 37s 5s/step - accuracy: 0.9168 - loss: 0.2145
Validation Accuracy: 0.92


# **ResNet50**

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

# Load and modify base model
base_model = ResNet50(input_shape=(128, 128, 3), include_top=False, weights='imagenet')
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(512, activation='relu', kernel_regularizer=l2(0.001))(x)
x = Dropout(0.5)(x)
x = Dense(3, activation='softmax')(x)

model_resnet50 = Model(inputs=base_model.input, outputs=x)

for layer in base_model.layers:
    layer.trainable = False

model_resnet50.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model (using the existing train_generator and validation_generator)
history = model_resnet50.fit(
    train_generator,
    steps_per_epoch=2,
    epochs=100,
    validation_data=validation_generator,
    validation_steps=5
)

Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


2/2 ━━━━━━━━━━━━━━━━━━━━ 249s 115s/step - accuracy: 0.2604 - loss: 2.0371 - val_accuracy: 0.3500 - val_loss: 1.8574
Epoch 2/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 26s 26s/step - accuracy: 0.2917 - loss: 2.0964 - val_accuracy: 0.3636 - val_loss: 1.7310
Epoch 3/100
1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.3750 - loss: 1.8007

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


2/2 ━━━━━━━━━━━━━━━━━━━━ 31s 31s/step - accuracy: 0.3333 - loss: 1.8006 - val_accuracy: 0.4000 - val_loss: 1.6783
Epoch 4/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step - accuracy: 0.2604 - loss: 1.9011 - val_accuracy: 0.2955 - val_loss: 1.6239
Epoch 5/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 39s 37s/step - accuracy: 0.3750 - loss: 1.6882 - val_accuracy: 0.3500 - val_loss: 1.5341
Epoch 6/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step - accuracy: 0.3021 - loss: 1.6848 - val_accuracy: 0.4318 - val_loss: 1.4639
Epoch 7/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 29s 29s/step - accuracy: 0.3646 - loss: 1.5338 - val_accuracy: 0.3750 - val_loss: 1.4010
Epoch 8/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - accuracy: 0.3021 - loss: 1.5088 - val_accuracy: 0.4545 - val_loss: 1.3363
Epoch 9/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 25s 25s/step - accuracy: 0.3731 - loss: 1.4350 - val_accuracy: 0.4375 - val_loss: 1.3184
Epoch 10/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step - accuracy: 0.2292 - loss: 1.5149 - val_accuracy: 0.5000 - val_loss: 1.2224
Epoch 11/10

In [ ]:
loss, accuracy = model_resnet50.evaluate(validation_generator)
print(f"Validation Accuracy: {accuracy:.2f}")

7/7 ━━━━━━━━━━━━━━━━━━━━ 25s 3s/step - accuracy: 0.5437 - loss: 0.9916
Validation Accuracy: 0.54


# **MobileNetV2**

In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout # Import necessary layers
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

base_model = MobileNetV2(input_shape=(128, 128, 3), include_top=False, weights='imagenet')
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(3, activation='softmax')(x)

model_mobilenetv2 = Model(inputs=base_model.input, outputs=x)

for layer in base_model.layers:
    layer.trainable = False

model_mobilenetv2.compile(optimizer=Adam(),
                          loss='categorical_crossentropy',
                          metrics=['accuracy'])

In [ ]:
# Train the model (using the existing train_generator and validation_generator)
history = model_mobilenetv2.fit(
    train_generator,
    steps_per_epoch=2,
    epochs=100,
    validation_data=validation_generator,
    validation_steps=5
)

Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


2/2 ━━━━━━━━━━━━━━━━━━━━ 288s 148s/step - accuracy: 0.3542 - loss: 1.6566 - val_accuracy: 0.7125 - val_loss: 0.7184
Epoch 2/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 28s 27s/step - accuracy: 0.6146 - loss: 0.9699 - val_accuracy: 0.6136 - val_loss: 0.9510
Epoch 3/100


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


2/2 ━━━━━━━━━━━━━━━━━━━━ 38s 37s/step - accuracy: 0.7565 - loss: 0.6554 - val_accuracy: 0.7437 - val_loss: 0.5674
Epoch 4/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 12s 12s/step - accuracy: 0.7500 - loss: 0.4934 - val_accuracy: 0.7955 - val_loss: 0.6140
Epoch 5/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 49s 46s/step - accuracy: 0.8021 - loss: 0.5783 - val_accuracy: 0.8438 - val_loss: 0.4746
Epoch 6/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step - accuracy: 0.8958 - loss: 0.3068 - val_accuracy: 0.9318 - val_loss: 0.2061
Epoch 7/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 50s 46s/step - accuracy: 0.8438 - loss: 0.3926 - val_accuracy: 0.8188 - val_loss: 0.3818
Epoch 8/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 8s 7s/step - accuracy: 0.8229 - loss: 0.4579 - val_accuracy: 0.8409 - val_loss: 0.4380
Epoch 9/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 23s 23s/step - accuracy: 0.9375 - loss: 0.2115 - val_accuracy: 0.8938 - val_loss: 0.3282
Epoch 10/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 7s 6s/step - accuracy: 0.8438 - loss: 0.3634 - val_accuracy: 0.8409 - val_loss: 0.3690
Epoch 1

In [ ]:
loss, accuracy = model_mobilenetv2.evaluate(validation_generator)
print(f"Validation Accuracy: {accuracy:.2f}")

7/7 ━━━━━━━━━━━━━━━━━━━━ 33s 4s/step - accuracy: 0.9303 - loss: 0.2873
Validation Accuracy: 0.93
